In [1]:
import requests

def download_csv(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        
        with open(filename, 'wb') as file:
            file.write(response.content)
        
        print(f"CSV file downloaded successfully as '{filename}'")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the CSV file: {e}")

# Example usage
csv_url = "https://raw.githubusercontent.com/EwoutBergsma/deleteme/refs/heads/main/extracted_texts.csv"  # Replace with actual CSV URL
save_as = "extracted_texts.csv"
download_csv(csv_url, save_as)


CSV file downloaded successfully as 'extracted_texts.csv'


In [ ]:
import csv

def read_csv(filename):
    try:
        companies = []
        frontpage_texts = []
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader, None)  # skip the headers
            for row in reader:
                companies.append(row[0])
                frontpage_texts.append(row[1])
                print(row)  # Prints each row as a list
            return companies, frontpage_texts
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
csv_filename = "extracted_texts.csv"
companies, frontpage_texts = read_csv(csv_filename)



In [11]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]



In [ ]:
# Open a CSV file to write the data
with open("company_summary.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Write header row
    writer.writerow(["Company", "ChatGPT Summary"])
    
    # Loop through companies and their corresponding frontpage texts
    for company, frontpage_text in zip(companies, frontpage_texts):
        prompt = f"The text of the frontpage of a company will be provided, delimited by triple backticks. Please provide a one-sentence summary of the company, merely respond with the summary, do not reply with anything else. ```{frontpage_text}```"
        completion = get_completion(prompt)
        # Remove ".html" from the company name
        company_name = company.replace(".html", "")
        
        # Write the row to the CSV
        writer.writerow([company_name, completion])
        print(company_name, completion)